# Phase 2 - Data Preprocessing and Cleaning 

 Step 1 - Data Ingestion & Initial Cleaning

In [ ]:
import pandas as pd
import numpy as np

# 1. Loading the dataset
# Make sure this path is correct for your local machine
file_path = r"Datasets\integrated_traffic_dataset (1).csv"

try:
    df = pd.read_csv(file_path)
    print(f"Successfully loaded data from {file_path}")
    print(f"Original shape (rows, columns): {df.shape}")

    # 2. Remove all exact duplicate rows
    original_rows = len(df)
    df = df.drop_duplicates()
    print(f"Removed {original_rows - len(df)} duplicate rows.")
    print(f"Shape after removing duplicates: {df.shape}")

    # 3. Droping clearly redundant columns
    redundant_cols = [
        'accident_reference', 
        'accident_year'
    ]
    
    df = df.drop(columns=redundant_cols, errors='ignore')
    print(f"Dropped redundant columns. Final shape: {df.shape}")
    print("--- Remaining Columns ---")
    print(df.columns.to_list())

except FileNotFoundError:
    print(f" ERROR: File not found at {file_path}")
    print("Please double-check the file path and try again.")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully loaded data from C:\Users\ali01\Documents\Ali\Final year PJ\Execution\integrated_traffic_dataset (1).csv
Original shape (rows, columns): (20085, 24)
Removed 0 duplicate rows.
Shape after removing duplicates: (20085, 24)
Dropped redundant columns. Final shape: (20085, 24)
--- Remaining Columns ---
['record_id', 'Timestamp', 'Location', 'Latitude', 'Longitude', 'Day_of_Week', 'Hour_of_Day', 'Incident_Congestion', 'Weather_Condition', 'Road_Condition', 'Vehicle_Type', 'Vehicle_Damage', 'Accident_Severity', 'Accident_Reason', 'Driver_Violation', 'Number_of_Deaths', 'Number_of_Injuries', 'Summary_Road_Name', 'Daily_Traffic_Volume', 'Daily_Average_Speed', 'Daily_Congestion_Level', 'Daily_Incident_Reports', 'Daily_Public_Transport_Usage', 'Daily_Parking_Usage']


_________________________________________________________________________________________________________________________________
Step 2: Data Type & Error Correction.

In [10]:
import pandas as pd
import numpy as np

print("--- 1. Correcting Data Types ---")

# --- Convert Timestamp to datetime ---
# This is critical for all temporal analysis in Phase 3
try:
    # Replace 'Timestamp' if your column name is different
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    print("'Timestamp' column converted to datetime.")
except KeyError:
    print("ERROR: 'Timestamp' column not found. Please check the name.")
except Exception as e:
    print(f"⚠Warning: Could not convert 'Timestamp'. Error: {e}")

# --- Convert key numerical columns ---
# We force errors to 'NaN' (Not a Number)
numerical_cols = ['Latitude', 'Longitude', 'Number_of_Vehicles', 'Number_of_Casualties']
for col in numerical_cols:
    if col in df.columns:
        original_type = df[col].dtype
        df[col] = pd.to_numeric(df[col], errors='coerce')
        print(f"'{col}' column converted from {original_type} to numeric.")
    else:
        print(f"Info: Column '{col}' not found, skipping conversion.")

print("\n--- 2. Marking Invalid Placeholders as NaN ---")

# --- Mark invalid Speed_Limit values ---
# Your plan specifies -1 or 99 as invalid
if 'Speed_Limit' in df.columns:
    # First, ensure it's numeric (handles text errors)
    df['Speed_Limit'] = pd.to_numeric(df['Speed_Limit'], errors='coerce')
    
    # Now, replace the invalid placeholders
    invalid_speeds = [-1, 99] 
    original_nan_count = df['Speed_Limit'].isna().sum()
    df['Speed_Limit'] = df['Speed_Limit'].replace(invalid_speeds, np.nan)
    new_nan_count = df['Speed_Limit'].isna().sum()
    print(f"'Speed_Limit': Marked {new_nan_count - original_nan_count} invalid values as NaN.")
else:
    print("ℹInfo: 'Speed_Limit' column not found, skipping.")

# --- Mark invalid Geocoordinates ---
# Your plan mentions -1 or 0 as invalid
if 'Latitude' in df.columns:
    df['Latitude'] = df['Latitude'].replace([-1, 0], np.nan)
    print("'Latitude': Marked -1 and 0 as NaN.")
if 'Longitude' in df.columns:
    df['Longitude'] = df['Longitude'].replace([-1, 0], np.nan)
    print("'Longitude': Marked -1 and 0 as NaN.")

print("\n--- Data Types After Correction ---")
df.info()

print("\n--- Missing Value Report (Before Imputation) ---")
# This shows us exactly what we need to impute in the next step
print(df.isna().sum())

--- 1. Correcting Data Types ---
'Timestamp' column converted to datetime.
'Latitude' column converted from float64 to numeric.
'Longitude' column converted from float64 to numeric.
Info: Column 'Number_of_Vehicles' not found, skipping conversion.
Info: Column 'Number_of_Casualties' not found, skipping conversion.

--- 2. Marking Invalid Placeholders as NaN ---
ℹInfo: 'Speed_Limit' column not found, skipping.
'Latitude': Marked -1 and 0 as NaN.
'Longitude': Marked -1 and 0 as NaN.

--- Data Types After Correction ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20085 entries, 0 to 20084
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   record_id                     20085 non-null  int64         
 1   Timestamp                     20085 non-null  datetime64[ns]
 2   Location                      20085 non-null  object        
 3   Latitude                      200

_________________________________________________________________________________________________________________________________
Step 3: Advanced (Decision Tree) & Simple Imputation

In [25]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder

def impute_with_decision_tree(df, target_col, predictor_features):
    """
    Imputes a target column using Decision Tree Regression.
    Predictor features must be categorical for OneHotEncoding.
    """
    print(f"\n--- Advanced Imputation for '{target_col}' ---")
    
    # Ensures predictor features exist
    valid_predictors = [col for col in predictor_features if col in df.columns]
    if not valid_predictors:
        print(f" ERROR: No valid predictor features found for '{target_col}'. Skipping.")
        return df
    
    # Create a copy to avoid changing original data during fitting
    impute_df = df[valid_predictors + [target_col]].copy()
    
    # Filling NaNs in predictors with 'Unknown'
    for col in valid_predictors:
        impute_df[col] = impute_df[col].fillna('Unknown').astype(str)
        
    # 1. Prepare data
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Fit encoder on ALL predictor data (train + predict)
    encoder.fit(impute_df[valid_predictors])
    
    # 2. Split data
    train_data = impute_df.dropna(subset=[target_col])
    predict_data = impute_df[impute_df[target_col].isna()]
    
    if predict_data.empty:
        print(f" No missing '{target_col}' values to impute.")
        return df
        
    if train_data.empty:
        print(f" ERROR: No training data (known values) for '{target_col}'. Cannot impute.")
        return df
        
    X_train = encoder.transform(train_data[valid_predictors])
    y_train = train_data[target_col]
    X_predict = encoder.transform(predict_data[valid_predictors])
    
    # 3. Train model
    print(f"Training Decision Tree model for '{target_col}'...")
    dt_reg = DecisionTreeRegressor(random_state=42, max_depth=10) # max_depth to prevent overfitting
    dt_reg.fit(X_train, y_train)
    
    # 4. Predict and fill
    predicted_values = dt_reg.predict(X_predict)
    
    # Fill the original DataFrame
    df.loc[df[target_col].isna(), target_col] = predicted_values
    print(f" Successfully imputed {len(predicted_values)} missing '{target_col}' values.")
    
    return df

# --- 1. Impute Speed_Limit ---
speed_col_name = 'YOUR_ACTUAL_SPEED_COLUMN' # e.g., 'speed_limit' or 'SpeedLimit'

speed_predictors = ['Road_Type', 'Day_of_Week_Name', 'Time_of_Day']

if 'Timestamp' in df.columns and pd.api.types.is_datetime64_any_dtype(df['Timestamp']):
    df['Day_of_Week_Name'] = df['Timestamp'].dt.day_name()
    df['Hour_of_Day'] = df['Timestamp'].dt.hour
    
    def get_time_of_day(hour):
        if 5 <= hour < 12: return 'Morning'
        elif 12 <= hour < 17: return 'Afternoon'
        elif 17 <= hour < 21: return 'Evening'
        else: return 'Night'
    df['Time_of_Day'] = df['Hour_of_Day'].apply(get_time_of_day).fillna('Night')
else:
    print(" Warning: Timestamp features not available. Imputation accuracy may be low.")
    speed_predictors = ['Road_Type']

if speed_col_name in df.columns:
    df = impute_with_decision_tree(df, speed_col_name, speed_predictors)
else:
    print(f"ERROR: Column '{speed_col_name}' not in DataFrame. Skipping speed limit imputation.")

# --- 2. Impute Geocoordinates ---
geo_predictors = ['Police_Force', 'Local_Authority_(District)', 'Road_Type'] # Check these names too!

if 'Latitude' in df.columns:
    df = impute_with_decision_tree(df, 'Latitude', geo_predictors)
else:
    print(" ERROR: Column 'Latitude' not in DataFrame. Skipping imputation.")

if 'Longitude' in df.columns:
    df = impute_with_decision_tree(df, 'Longitude', geo_predictors)
else:
    print(" ERROR: Column 'Longitude' not in DataFrame. Skipping imputation.")


# --- Part 3B: Simple Imputation ---
print("\n--- Simple Imputation for Remaining Columns ---")

categorical_cols = ['Driver_Violation', 'Vehicle_Damage', 'Weather_Conditions', 'Road_Surface_Conditions']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')
        print(f" Filled missing '{col}' values with 'Unknown'.")

numerical_count_cols = ['Number_of_Vehicles', 'Number_of_Casualties']
for col in numerical_count_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)
        print(f" Filled missing '{col}' values with median ({median_val}).")

# --- Final Check ---
print("\n--- Missing Value Report (After Imputation) ---")
print(df.isna().sum())

ERROR: Column 'YOUR_ACTUAL_SPEED_COLUMN' not in DataFrame. Skipping speed limit imputation.

--- Advanced Imputation for 'Latitude' ---
 ERROR: No valid predictor features found for 'Latitude'. Skipping.

--- Advanced Imputation for 'Longitude' ---
 ERROR: No valid predictor features found for 'Longitude'. Skipping.

--- Simple Imputation for Remaining Columns ---
 Filled missing 'Driver_Violation' values with 'Unknown'.
 Filled missing 'Vehicle_Damage' values with 'Unknown'.

--- Missing Value Report (After Imputation) ---
record_id                       0
Timestamp                       0
Location                        0
Latitude                        0
Longitude                       0
Day_of_Week                     0
Hour_of_Day                     0
Incident_Congestion             0
Weather_Condition               0
Road_Condition                  0
Vehicle_Type                    0
Vehicle_Damage                  0
Accident_Severity               0
Accident_Reason             

_________________________________________________________________________________________________________________________________
Step 4: Feature Engineering & Final Validation

In [28]:
import pandas as pd
import numpy as np
print("--- 4A: Engineering Temporal Features (for EDA) ---")

if 'Timestamp' in df.columns and pd.api.types.is_datetime64_any_dtype(df['Timestamp']):
    # I already created Hour_of_Day and Day_of_Week_Name in Step 3
    # Now I'll create the remaining temporal features I need
    
    # I'll create a numeric day-of-week (0=Monday) for sorting
    if 'Day_of_Week_Num' not in df.columns:
        df['Day_of_Week_Num'] = df['Timestamp'].dt.dayofweek
    
    # I'll extract the Month
    if 'Month' not in df.columns:
        df['Month'] = df['Timestamp'].dt.month
    
    # I'll create a binary 'Is_Weekend' feature
    if 'Is_Weekend' not in df.columns:
        df['Is_Weekend'] = df['Day_of_Week_Num'].isin([5, 6]).astype(int)
    
    print("My new temporal features are: 'Day_of_Week_Num', 'Month', 'Is_Weekend'")
else:
    print("Warning: 'Timestamp' column not found or not datetime. Skipping temporal features.")

# --- 4B: Engineering Discretized Features (for Apriori) ---
print("\n--- 4B: Engineering Discretized Features (for Apriori) ---")

# 1. I need to discretize 'Speed_Limit'
# !!! IMPORTANT: I must replace 'YOUR_ACTUAL_SPEED_COLUMN' with my real column name
speed_col_name = 'YOUR_ACTUAL_SPEED_COLUMN' # e.g., 'Speed_Limit'

if speed_col_name in df.columns:
    # I'll create string-based bins for my Apriori analysis
    speed_bins = [0, 30, 50, 70, 100] # Bins: 0-29, 30-49, 50-69, 70-99
    speed_labels = ['Speed_0-29', 'Speed_30-49', 'Speed_50-69', 'Speed_70-99']
    
    df['Speed_Limit_Bin'] = pd.cut(df[speed_col_name],
                                 bins=speed_bins,
                                 labels=speed_labels,
                                 right=False) # 'right=False' means [0-30)
    
    # I'll convert this to a string and fill any NaNs
    df['Speed_Limit_Bin'] = df['Speed_Limit_Bin'].astype(str).replace('nan', 'Speed_Unknown')
    print(f"Created 'Speed_Limit_Bin' from '{speed_col_name}'.")
else:
    print(f"Warning: Column '{speed_col_name}' not found. Skipping 'Speed_Limit_Bin'.")

# 2. I'll ensure all my other Apriori features are clean strings
# (I mostly did this in Step 3, but this confirms it)
apriori_cols = ['Accident_Severity', 'Time_of_Day', 'Day_of_Week_Name', 
                'Light_Conditions', 'Weather_Conditions', 'Road_Surface_Conditions', 
                'Junction_Control', 'Vehicle_Damage']

print("Ensuring all Apriori columns are string type...")
for col in apriori_cols:
    if col in df.columns:
        df[col] = df[col].astype(str)
    else:
        print(f"Warning: Apriori column '{col}' not found.")


# --- 4C: Final Validation & Save ---
print("\n\n--- FINAL DATA QUALITY REPORT (Phase 2 Complete) ---")

print("\n--- Final Data Info & Types ---")
df.info()

print("\n--- Final Missing Values Check ---")
final_missing = df.isna().sum()
if final_missing.sum() == 0:
    print("My dataset has no missing values remaining.")
else:
    print("Warning: Missing values are still present:")
    print(final_missing[final_missing > 0])

# 4. I'll save the fully cleaned and feature-engineered dataset
cleaned_file_name = "cleaned_for_phase_3.csv"
df.to_csv(cleaned_file_name, index=False)
print(f"\nPhase 2 is complete. My cleaned data is saved to '{cleaned_file_name}'")

--- 4A: Engineering Temporal Features (for EDA) ---
My new temporal features are: 'Day_of_Week_Num', 'Month', 'Is_Weekend'

--- 4B: Engineering Discretized Features (for Apriori) ---
Ensuring all Apriori columns are string type...


--- FINAL DATA QUALITY REPORT (Phase 2 Complete) ---

--- Final Data Info & Types ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20085 entries, 0 to 20084
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   record_id                     20085 non-null  int64         
 1   Timestamp                     20085 non-null  datetime64[ns]
 2   Location                      20085 non-null  object        
 3   Latitude                      20085 non-null  float64       
 4   Longitude                     20085 non-null  float64       
 5   Day_of_Week                   20085 non-null  object        
 6   Hour_of_Day                   20085 non